<a href="https://colab.research.google.com/github/patrickctrf/IA024_2022S2/blob/main/ex03/patrick_ferreira/ex03_patrick_ferreira_175480.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
nome = 'Patrick de Carvalho Tavares Rezende Ferreira'
print(f'Meu nome é {nome}')

Meu nome é Patrick de Carvalho Tavares Rezende Ferreira


## Instruções

- Treinar uma rede neural de duas camadas como classificador binário na tarefa de análise de sentimentos usando dataset IMDB usando TF-IDF como entrada.

Deve-se implementar o laço de treinamento e validação da rede neural.

Neste exercício usaremos o IMDB com 20k exemplos para treino, 5k para desenvolvimento e 25k para teste.

# Importando os pacotes necessários

In [3]:
import collections
import os
import random
import re
import torch
import numpy as np

# Verificando se a GPU está disponível

In [4]:
if torch.cuda.is_available(): 
   dev = "cuda:0"
   print(torch. cuda. get_device_name(dev))
else:
   dev = "cpu" 
print(dev)
device = torch.device(dev)

NVIDIA GeForce 940MX
cuda:0


## Preparando Dados

Primeiro, fazemos download do dataset:

In [5]:
!wget -nc http://files.fast.ai/data/aclImdb.tgz 
!tar -xzf aclImdb.tgz

File ‘aclImdb.tgz’ already there; not retrieving.



## Carregando o dataset

Criaremos uma divisão de treino (80%) e validação (20%) artificialmente.

Nota: Evitar de olhar ao máximo o dataset de teste para não ficar enviseado no que será testado. Em aplicações reais, o dataset de teste só estará disponível no futuro, ou seja, é quando o usuário começa a testar o seu produto.

In [6]:
def load_texts(folder):
    texts = []
    for path in os.listdir(folder):
        with open(os.path.join(folder, path)) as f:
            texts.append(f.read())
    return texts

x_train_pos = load_texts('aclImdb/train/pos')
x_train_neg = load_texts('aclImdb/train/neg')
x_test_pos = load_texts('aclImdb/test/pos')
x_test_neg = load_texts('aclImdb/test/neg')

x_train = x_train_pos + x_train_neg
x_test = x_test_pos + x_test_neg
y_train = [True] * len(x_train_pos) + [False] * len(x_train_neg)
y_test = [True] * len(x_test_pos) + [False] * len(x_test_neg)

# Embaralhamos o treino para depois fazermos a divisão treino/valid.
c = list(zip(x_train, y_train))
random.shuffle(c)
x_train, y_train = zip(*c)

n_train = int(0.8 * len(x_train))

x_valid = x_train[n_train:]
y_valid = y_train[n_train:]
x_train = x_train[:n_train]
y_train = y_train[:n_train]

print(len(x_train), 'amostras de treino.')
print(len(x_valid), 'amostras de desenvolvimento.')
print(len(x_test), 'amostras de teste.')

print('3 primeiras amostras treino:')
for x, y in zip(x_train[:3], y_train[:3]):
    print(y, x[:100])

print('3 últimas amostras treino:')
for x, y in zip(x_train[-3:], y_train[-3:]):
    print(y, x[:100])

print('3 primeiras amostras validação:')
for x, y in zip(x_valid[:3], y_test[:3]):
    print(y, x[:100])

print('3 últimas amostras validação:')
for x, y in zip(x_valid[-3:], y_valid[-3:]):
    print(y, x[:100])

20000 amostras de treino.
5000 amostras de desenvolvimento.
25000 amostras de teste.
3 primeiras amostras treino:
False Okay, first off, Seagal's voice is dubbed over for like 50% of the film... Why? Because apparently t
True This movie was featured on a very early episode of Mystery Science Theater 3000, but when I see this
True This low-budget erotic thriller that has some good points, but a lot more bad one. The plot revolves
3 últimas amostras treino:
False I thought they should have called this movie "Whites" instead of "Heights". Godawful...the kind of f
False "Serum" starts out with credits that are quite reminiscent of the "Re-animator" movies, and it owes 
False *spoliers* do not read any further if you haven't seen this movie<br /><br />Picking up after the de
3 primeiras amostras validação:
True Typical 90's comedy, situational comedy similar to our modern day "My Family". Thatcher being the he
True I like Goldie Hawn and wanted another one of her films, so when I saw Protoc

### Definindo funções de manipulação de texto.

In [11]:
from typing import List

import re
import string


def tokenize(text: str):
    """
    Convert string to a list of tokens (i.e., words).
    This function lower cases everything and removes punctuation.
    """

    return re.sub('[' + string.punctuation + ']', '', text).lower().split()

from collections import Counter


def create_vocab(texts: List[str], max_tokens: int):
    """
    Returns a dictionary whose keys are tokens and values are token ids (from 0 to max_tokens - 1).
    """

    tokens = []

    for t in texts:
        tokens.extend(tokenize(t))

    return dict(Counter(tokens).most_common(max_tokens))

def concatenate_list_of_str(texts: List[str]):
    return "".join(texts)

### Criando classe do dataset

### Testando dataset


In [13]:
x_assert = ["It is going to rain today.",
            "Today I am not going outside.",
            "I am going to watch the season premiere."]

y_assert = [False, True, True]
